In [7]:
from pathlib import Path
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Encuentra la carpeta /data sin importar dónde esté el .ipynb
here = Path.cwd()
candidates = [here, *here.parents]
data_dir = None
for c in candidates:
    d = c / "data"
    if d.exists():
        data_dir = d
        break
assert data_dir is not None, "No encuentro carpeta /data. Pon el notebook dentro del repo."

data_dir

WindowsPath('C:/Users/pablo/Downloads/SEMESTRE10/ANÁLISIS DE DATOS/Parcial 2/data')

In [8]:
# === Tablas visibles de TODAS las temporadas en /data ===
from pathlib import Path
import glob
import pandas as pd
from IPython.display import display, Markdown

REQ = {"HomeTeam","AwayTeam","FTHG","FTAG"}

# 1) Localiza carpeta /data (usa tu ruta de Windows y fallback a ./data)
candidatas = [
    Path(r"C:/Users/pablo/Downloads/SEMESTRE10/ANÁLISIS DE DATOS/Parcial 2/data"),
    Path.cwd() / "data",
]
data_dir = next((d for d in candidatas if d.exists()), None)
assert data_dir is not None, "No encuentro la carpeta /data. Ajusta la ruta arriba."

def leer_sp1_seguro(path: Path) -> pd.DataFrame:
    """Lee CSV intentando utf-8 y latin-1; valida columnas requeridas."""
    for enc in ("utf-8", "latin-1"):
        try:
            df = pd.read_csv(path, encoding=enc)
            break
        except Exception:
            df = None
    if df is None:
        raise RuntimeError(f"No pude leer {path.name} (codificación).")
    if not REQ.issubset(df.columns):
        raise ValueError(f"{path.name} debe tener columnas: {REQ}")
    return df

def tabla_liga(df_matches: pd.DataFrame) -> pd.DataFrame:
    """Tabla ordenada por puntos, luego GD y GF (desc)."""
    home = df_matches[["HomeTeam","FTHG","FTAG"]].rename(columns={"HomeTeam":"team","FTHG":"GF","FTAG":"GA"})
    away = df_matches[["AwayTeam","FTAG","FTHG"]].rename(columns={"AwayTeam":"team","FTAG":"GF","FTHG":"GA"})
    full = pd.concat([home, away], ignore_index=True)
    agg = full.groupby("team").agg(GF=("GF","sum"), GA=("GA","sum"), MP=("GF","count")).reset_index()
    agg["GD"] = agg["GF"] - agg["GA"]
    # puntos: local
    ph = df_matches.assign(p=(df_matches.FTHG>df_matches.FTAG)*3 + (df_matches.FTHG==df_matches.FTAG)*1)\
                   .groupby("HomeTeam")["p"].sum().rename_axis("team").rename("p_home").reset_index()
    # puntos: visita
    pa = df_matches.assign(p=(df_matches.FTAG>df_matches.FTHG)*3 + (df_matches.FTAG==df_matches.FTHG)*1)\
                   .groupby("AwayTeam")["p"].sum().rename_axis("team").rename("p_away").reset_index()
    pts = pd.merge(ph, pa, on="team", how="outer").fillna(0.0)
    pts["points"] = pts["p_home"] + pts["p_away"]
    tbl = agg.merge(pts[["team","points"]], on="team", how="left").fillna({"points":0})
    tbl = tbl.sort_values(["points","GD","GF"], ascending=[False, False, False]).reset_index(drop=True)
    tbl["rank"] = tbl.index + 1
    # columnas ordenadas
    return tbl[["rank","team","points","GF","GA","GD","MP"]]

# 2) Recorre todas las temporadas y muestra tablas
archivos = sorted(glob.glob(str(data_dir / "Season *.csv")))
assert archivos, f"No encontré archivos 'Season *.csv' en {data_dir}"

for ruta in archivos:
    p = Path(ruta)
    try:
        dfm = leer_sp1_seguro(p)
        tbl = tabla_liga(dfm)
        display(Markdown(f"### {p.stem}  —  Equipos: {len(tbl)}"))
        display(tbl)
    except Exception as e:
        display(Markdown(f"**[WARN]** Saltando `{p.name}`: {e}"))


### Season 1993-1994  —  Equipos: 20

,rank,team,points,GF,GA,GD,MP
0,1,Barcelona,81,91.0,42.0,49.0,38
1,2,La Coruna,78,54.0,18.0,36.0,38
2,3,Zaragoza,65,71.0,47.0,24.0,38
3,4,Real Madrid,64,61.0,49.0,12.0,38
4,5,Ath Bilbao,59,61.0,47.0,14.0,38
5,6,Sevilla,57,56.0,42.0,14.0,38
6,7,Valencia,54,55.0,50.0,5.0,38
7,8,Santander,53,44.0,42.0,2.0,38
8,9,Tenerife,51,50.0,57.0,-7.0,38
9,10,Sp Gijon,50,42.0,57.0,-15.0,38


### Season 1994-1995  —  Equipos: 20

,rank,team,points,GF,GA,GD,MP
0,1,Real Madrid,78,76.0,29.0,47.0,38
1,2,La Coruna,71,68.0,32.0,36.0,38
2,3,Barcelona,64,60.0,45.0,15.0,38
3,4,Betis,61,46.0,25.0,21.0,38
4,5,Zaragoza,61,56.0,51.0,5.0,38
5,6,Sevilla,59,55.0,41.0,14.0,38
6,7,Ath Bilbao,58,39.0,42.0,-3.0,38
7,8,Espanol,57,51.0,35.0,16.0,38
8,9,Oviedo,52,45.0,42.0,3.0,38
9,10,Valencia,51,53.0,48.0,5.0,38


### Season 1995-1996  —  Equipos: 22

...

,rank,team,points,GF,GA,GD,MP
0,1,Real Madrid,86,80,31,49,38
1,2,Barcelona,73,68,38,30,38
2,3,Ath Madrid,71,65,43,22,38
3,4,Sevilla,70,53,30,23,38
4,5,Betis,65,62,40,22,38
5,6,Sociedad,62,40,37,3,38
6,7,Villarreal,59,63,37,26,38
7,8,Ath Bilbao,55,43,36,7,38
8,9,Valencia,48,48,53,-5,38
9,10,Osasuna,47,37,51,-14,38


### Season 2022-2023  —  Equipos: 20

,rank,team,points,GF,GA,GD,MP
0,1,Barcelona,88,70,20,50,38
1,2,Real Madrid,78,75,36,39,38
2,3,Ath Madrid,77,70,33,37,38
3,4,Sociedad,71,51,35,16,38
4,5,Villarreal,64,59,40,19,38
5,6,Betis,60,46,41,5,38
6,7,Osasuna,53,37,42,-5,38
7,8,Ath Bilbao,51,47,43,4,38
8,9,Mallorca,50,37,43,-6,38
9,10,Girona,49,58,55,3,38


### Season 2023-2024  —  Equipos: 20

,rank,team,points,GF,GA,GD,MP
0,1,Real Madrid,95,87,26,61,38
1,2,Barcelona,85,79,44,35,38
2,3,Girona,81,85,46,39,38
3,4,Ath Madrid,76,70,43,27,38
4,5,Ath Bilbao,68,61,37,24,38
5,6,Sociedad,60,51,39,12,38
6,7,Betis,57,48,45,3,38
7,8,Villarreal,53,65,65,0,38
8,9,Valencia,49,40,45,-5,38
9,10,Alaves,46,36,46,-10,38
